# Creation of Land Use Change Tables from Generated Google Earth Assets


In [1]:
import ee
import geemap
import geemap.ml as ml
from ipygee import chart as chart
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import pymannkendall as mk
import xarray as xr
import os
# Import date class from datetime module
from datetime import datetime as dt
import datetime
import pytz

today = dt.today()
print("Today is: ", today)

Today is:  2023-05-19 20:04:58.061918


# GEE Authentication 
 
 ### Paste the Authetication code into the box below if prompted to save token
 
 
 (press enter to save token)


In [2]:
# ee.Authenticate()

In [3]:
geemap.ee_initialize()

### New version control of inputs and outputs

* best to check that catchment hydroclimatic information is indeed the most reliable/latest available

In [4]:
# set the path and version of the input data
p = '..'
version = 'Version_3_20230303'

# read in the list of catchment IDs from the input csv file
l = pd.read_csv(f"{p}/Inputs/{version}/USA.csv", index_col=0)


# display the list of catchment IDs and convert it to a Python list
names = l.index.tolist()

# names = [22001, 23004, 71004, 79002, 94001]

# print the number of catchments and their IDs
print(f'{len(names)} catchments processed for hydroclimatic variables:\n \n{names}')

169 catchments processed for hydroclimatic variables:
 
[1411300, 1484100, 1485500, 1486000, 1487000, 1491000, 1580000, 1583500, 1591400, 1632900, 1638480, 1639500, 1644000, 1658500, 1664000, 1666500, 1667500, 1669000, 2013000, 2014000, 2017500, 2018000, 2027000, 2028500, 2038850, 2046000, 2051000, 2051500, 2053200, 2053800, 2055100, 2056900, 2059500, 2064000, 2065500, 2069700, 2070000, 2074500, 2077200, 2081500, 2082950, 2102908, 2111180, 2111500, 2112120, 2112360, 2118500, 2125000, 2128000, 2143000, 2143040, 2149000, 2152100, 2177000, 2178400, 2202600, 2212600, 2221525, 2342933, 2349900, 2395120, 2408540, 2422500, 2430085, 2430615, 2450250, 3140000, 3144000, 3159540, 3161000, 3164000, 3165000, 3170000, 3173000, 3213700, 3237280, 3237500, 3238500, 3241500, 3281500, 3285000, 3291780, 3300400, 3340800, 3346000, 3357350, 3364500, 3366500, 3368000, 3384450, 3439000, 3455500, 3456500, 3460000, 3471500, 3473000, 3479000, 3488000, 3498500, 3504000, 3574500, 3604000, 4196800, 5487980, 5489000


### Load the JS Module

The custom JS module takes the difficult javascript functions that do not translate well to python, and makes them callable in the notebook environment. 

Current version 6 improves the classifier by using a weighted training strategy.

This custom module borrows some functions from the LandTrendr module developed by Justin Braaten (Google) which is classified under an apache license i.e. free for use). The adaptation begins from landTrendr version 0.2 which incorporated Landsat Collection 2, removing the need for regression coefficients between sensors developed by roy et. al.

In [5]:
oeel = geemap.requireJS()

Map = geemap.Map()

ltgee = geemap.requireJS(r'../JS_module/Adapted_LT_v8.2.js')

ltgee.availability  #all functions within the javascript module

IMPORTANT! Please be advised:
- This version of the Adapted_LT.js modules
  uses some code adapted from the aut/or: @author Justin Braaten (Google) * @author Zhiqiang Yang (USDA Forest Service) * @author Robert Kennedy (Oregon State University)
The latest edits to this code occur: 08/03/2023 for the adaptation efforts by @Mike OHanrahan (TU DELFT MSc research)


{'version': 'string',
 'buildSensorYearCollection': 'function',
 'getSRcollection': 'function',
 'getCombinedSRcollection': 'function',
 'buildSRcollection': 'function',
 'getCollectionIDlist': 'function',
 'countClearViewPixels': 'function',
 'buildClearPixelCountCollection': 'function',
 'removeImages': 'function',
 'LAIcol': 'function',
 'calcIndex': 'function',
 'standardize': 'function',
 'transformSRcollection': 'function',
 'createTrainingImage': 'function',
 'addTerrainBand': 'function',
 'genGCP': 'function',
 'classifier': 'function',
 'classArea': 'function',
 'imcolFromAsset': 'function',
 'imcolFromAssetHILDA': 'function',
 'unionCollections': 'function'}

## Initiate With a Shapefile

This notebook assumes the user has a shapefile saved as an asset on their GEE, the assets used in the CATAPUCII project will be made publicly available in the @mohanrahan repository


In [36]:
# Directory where assets are stored
asset_dir = 'projects/mohanrahan/assets'

# Asset ID for catchment boundaries
catchment_asset = 'CATAPUCII_Catchments/CAMELS_USA_catchment_boundaries'

# Name of the dataset
dataset = 'CAMELS_US'

# Column string to identify catchments
col_string  = 'hru_id'

# Coordinate reference system, GB is british national grid
crs = 'EPSG:4269'

# Figure number for plotting
fignum = 0

# RGB visualization settings for Landsat imagery
RGB_VIS = {'bands':['B3','B2','B1'], 'min':0, 'max':1.5e3}

#Classified image visualisation
lc_vis = {'bands':['landcover'], 'min':1, 'max':5, 'palette':['#E6004D', '#FFFFA8', '#80FF00', '#A6A6FF', '#00CCF2']}

# Start and end years for Landsat data collection
startYear = 1899
endYear = 2019

# Start and end days for Landsat data collection
startDay = '06-20'
endDay = '08-31'

# List of images to be masked from Landsat collection
maskThese = ['cloud', 'shadow', 'snow',]

# List of bands to include in Landsat collection
bandList = ["B1", "B2", "B3", "B4", "B5", "B7", 
           'NBR', 'NDMI', 'NDVI', 'NDSI', 'EVI','GNDVI', 
           'TCB', 'TCG', 'TCW', 'TCA', 'NDFI',] 

## The Table Data


In [37]:
# Define the feature collection from asset directory and catchment asset name
table = ee.FeatureCollection(f"{asset_dir}/{catchment_asset}")

# Define a function to calculate the area of each geometry in square kilometer
def set_area_km2(feature):
    '''
    Calculate the area of each geometry in square kilometer
    '''
    area = feature.geometry().area().divide(1000*1000)
    setting = feature.set('area_km2', area)
    return setting

# Define a function to calculate the area of each geometry in pixels
def set_area_pixel(feature):
    '''
    Calculate the area of each geometry in pixels
    '''
    aoi = feature.geometry()
    area = ee.Image.pixelArea().divide(1e6).clip(aoi).select('area').reduceRegion(**{
        'reducer':ee.Reducer.sum(),
        'geometry':aoi,
        'scale':30,
        'crs':crs,
        'maxPixels':1e13,
        'bestEffort':True,
        }).get('area')
    setting = feature.set('pixel_area', area)
    return setting

# Define a function to set the system ID as a column
def set_id(feature):
    '''
    Set the system ID as a column
    '''
    getting_name = ee.String(feature.get('system:index'))
    setting_id = feature.set({'system_index':getting_name,})
    return setting_id

# Calculate the area of each geometry and set ID column and pixel area column
table_area = table.map(set_area_km2).map(set_id).map(set_area_pixel)

# Filter out geometries with area_km2 equal to zero and sort by area from largest to smallest
Filtered_Sorted = table_area.filter(ee.Filter.inList(col_string, ee.List(names))).filter(ee.Filter.gt('area_km2', 0)).sort('area_km2', True)


# Convert the sorted table to a Pandas dataframe and set the index to 'system_index'
sys_index = Filtered_Sorted.aggregate_array('system_index').getInfo()#.set_index(['system_index'])
area = np.array(ee.Array(Filtered_Sorted.aggregate_array('AREA')).toFloat().getInfo()) *1e-6
sys_id = Filtered_Sorted.aggregate_array('hru_id').getInfo()

print(sys_id)

[1484100, 6879650, 1486000, 2102908, 1658500, 2038850, 3357350, 3368000, 2430615, 3237280, 2055100, 7195800, 2430085, 1591400, 2143040, 3140000, 3291780, 3455500, 1669000, 1411300, 2395120, 5591550, 7335700, 7196900, 3384450, 1485500, 3165000, 6803510, 2349900, 2077200, 3460000, 2111180, 3504000, 3456500, 2051000, 2125000, 5593900, 2178400, 2152100, 1583500, 3241500, 3439000, 1487000, 2212600, 3471500, 2149000, 2112360, 5503800, 5593575, 2143000, 2069700, 5595730, 2111500, 1638480, 3479000, 2027000, 3364500, 2450250, 2028500, 1580000, 1632900, 7340300, 2065500, 5592050, 1639500, 2128000, 7180500, 5507600, 8082700, 2070000, 2017500, 2053800, 2074500, 2342933, 2046000, 6911900, 1491000, 2056900, 6803530, 7167500, 6921200, 2112120, 3340800, 3144000, 2014000, 2118500, 7145700, 3159540, 6889200, 2064000, 2081500, 2013000, 6910800, 7261000, 2082950, 1666500, 6903400, 2059500, 2221525, 7315200, 7184000, 2177000, 3161000, 5508805, 3238500, 3488000, 2053200, 2422500, 4196800, 2202600, 6918460, 

In [38]:
aoi = Filtered_Sorted.geometry().bounds()

imCol = ltgee.imcolFromAssetHILDA(startYear, endYear, aoi, 'CAMELS_USA', "projects/mohanrahan/assets/HILDA")

Map = geemap.Map()
# Map.setOptions('')
Map.addLayer(ee.Image(imCol.mode()).clip(aoi), {'bands':['b1'], 'min':1, 'max':99, 'palette':['#E6004D', '#FFFFA8', '#80FF00', '#A6A6FF', '#00CCF2']}, 'Mode of Classes')
# Map.addLayer(aoi, {'color': 'green'}, 'green: Included')
# Map.addLayer(naoi, {'color':'red'}, 'red: Not Included')
Map.centerObject(aoi, 10)


Map

Map(center=[36.61332540136129, -88.30813539565372], controls=(WidgetControl(options=['position', 'transparent_…

In [39]:
def extractArea(item):
    
    '''
    Method borrowed from https://code.earthengine.google.co.in/9c45ff677c46eae08952831de02bfb40
    Article: https://spatialthoughts.com/2020/06/19/calculating-area-gee/
    '''
    
    areaDict = ee.Dictionary(item)
    classNumber = ee.Number(areaDict.get('b1')).format()
    area = ee.Number(areaDict.get('sum')).divide(1e6)
    return ee.List([classNumber, area])

def classArea(classified_image, scale, aoi):
    '''
    This function takes the pixel areas represented by each class the landsat scale is 30m but,
    nominal scale of image is 111000m after medoid compositing
    '''
    
    areaImage = ee.Image.pixelArea().addBands(classified_image)
    
    areas = areaImage.reduceRegion(**{
            'reducer':ee.Reducer.sum().group(**{'groupField':1, 'groupName':'b1'}),
            'geometry':aoi,
            'scale':scale,
            'maxPixels':1e10,
            'bestEffort':True,
    })
    
    classAreas = ee.List(areas.get('groups'))
    
    classAreasLists = classAreas.map(extractArea)
    
    return classAreasLists

def dateToMs(year):
    datetime_obj = datetime.datetime(year, 8, 31, 22, 0, 0)
    return datetime_obj

def dataframeAreas(i, yc, aoi, classified, trainingClassImage, ms, classImageYear, name, accuracy, pixArea):

    ls1 = pd.DataFrame(classArea(classified, 1113, aoi).getInfo(), columns=['class', 'area_H'])
    # ls2 = pd.DataFrame(classArea(trainingClassImage, 1000, aoi).getInfo(), columns=['class', 'area_CORINE'])

    merged = ls1

    merged['image_date'] = ms
    pivoted = merged.pivot(index='image_date', columns='class', values=['area_H'])
    pivoted['training', 'year_trained'] =  classImageYear

    pivoted['catchment', 'area'] = pixArea
    
    # pivoted['area_RF', '6'] = pivoted.catchment.area - pivoted.iloc[0, 6:10].sum() 
    pivoted['catchment', 'name '] = name
    # pivoted['testing', 'accuracy'] = accuracy
    # pivoted['ind'] = str(i)+'_'+str(yc)
    pivoted.fillna(0)
    # print(pivoted)
    return pivoted

## Running Module over the Shapefile

1. The geometries are called by their system indices (sys_index) updating the 'aoi' and running the process over any  using the indices included in the 
2. The image collection is generated per shapefile and then returns the decadal mean of each index

# TODO:

- Redefine the methodology of reduction. Using chart --> dataframe --> join all dataframes is redundant an probably very slow

In [40]:
classLoopParams = {'dataset':'CORINE',    #training dataset, no other than corine currently supported
               'trainingClassLevel':1, #classLevel determines the level of corine class simplification
               'customClassLevels':None,   #can provide some custom levels, not fully tested
               'numClasses':5,            #if trainingClassLevel is 1 then there are 5 classes, level is 2 then there are 15, 3 is 44. (CORINE land cover class grouping)
               'tileScale':2,            #tileScale higher number reduces likelihood of classifier running into a memory limit
               'year_classified': np.arange(2019, 2020),   # classification year is , for classifiers saved, the same as the years available in the training dataset
              }

output_folder = f'../Outputs/{dataset}/class_areas_from_asset_HILDA/'

if not os.path.exists(output_folder):
    print(f'{output_folder} created')
    os.makedirs(output_folder)



In [49]:
index_broken = '0000000000000000010a'
year_broken = 1899

US_decades = ('1981 - 1989', '1990 - 1999', '2000 - 2009')

In [50]:
t0 = dt.today()

scale =1113

print(f'begin loop: {t0}')

classArea_df = pd.DataFrame()


index = sys_index.index(index_broken)


# Slice the list based on the length of the string
slice_start = index
slice_broken = index + sys_index.count(index_broken) -1



for i, ind in enumerate(sys_index[0:]): # Loop through all indices in the system index
    
    year_range = [1899, 1930, 1950, 1960, 1970, 1979, 1980,] + list(np.arange(1981, 2020))
    
    if ind == index_broken:
        startYear = year_broken
        year_range = year_range[year_range.index(startYear):]  # Update year_range starting from startYear
        print('resuming from broken')
    else:
        year_range = year_range
    
    print(f'{slice_broken+i} / {len(sys_index)}')
    
    
    name = ind
    feat = Filtered_Sorted.filterMetadata('system:index', 'equals', ind).first()
    
    aoi = Filtered_Sorted.filterMetadata('system:index', 'equals', ind).first().geometry()
    pix_area = feat.get('pixel_area').getInfo() # Get area of the featur
    
    t1 = dt.today()
    
    if dataset == 'CAMELS_US':
        '''
        
        '''
        for j, yc in enumerate(year_range):
            
            print('begin', dataset, name, yc)
            
            #the image from the collection that we want to classify
            classified = imCol.filterDate(str(yc+100)+'-'+startDay, str(yc+101)+'-'+endDay).first()
        
            ms = dateToMs(yc)
           
            print(ms)
            
            try:
                df = dataframeAreas(i, yc, aoi, classified, ee.Image.constant(ee.Number(1)), ms, yc, f'{name}', 1, pix_area)

                df.to_excel(f'{output_folder}{name}_{yc}_HILDA.xlsx')

                classArea_df = classArea_df.append(df)
            except EEException as e:
                print(e)
                continue

    else:
        print('classification routine for this dataset is not yet provided for')
    
    t4 = dt.today()
    
    print(f'step2: Done: {t4}, time taken: {t4-t1}')
    
    print(f'\nCatchment: {name}, total time: {t4-t1}\n---------------')
    


tfinal = dt.today()

print(f'END LOOP: Full routine finished: {tfinal} \nTime taken: {tfinal-t0}')

begin loop: 2023-05-19 21:01:13.764257
resuming from broken
0 / 169
begin CAMELS_US 0000000000000000010a 1899
1899-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1930
1930-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1950
1950-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1960
1960-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1970
1970-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1979
1979-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1980
1980-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1981
1981-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1982
1982-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1983
1983-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1984
1984-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1985
1985-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1986
1986-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1987
1987-08-31 22:00:00
begin CAMELS_US 0000000000000000010a 1988
1988-08-31 22:00:00
be